# Design E-Ticketing System

Let's design an online E-ticketing system that sells movie tickets.

A movie ticket booking system provides its customer the ability to purchase theatre seats online. E-ticketing systems allows the customer to browse movies currently being played and to book available seats, anywhere anytime.

## 1. Requirements and System goals

### Functional Requirements
- The service should list different cities where its affiliated cinemas are located.
- When user selects a city, the service should display movies released in that particular city.
- When user selects movie, the service should display the cinemas running the movie plus available show times.
- Users should be able to book a show at a cinema and book tickets.
- The service should be able to show the user the seating arrangement of the cinema hall. The user should be able to select multiple seats according to the preference.
- The user should be able to distinguish between available seats from booked ones.
- Users should be able to put a hold on the seat (for 5 minutes) while they make payments.
- Users should be able to wait if there is a chance that the seats might become available (When holds by other users expire).
- Waiting customers should be serviced in a fair, first come, first serve manner.

### Non-Functional Requirements
- The service should be highly concurrent. There will be multiple booking requests for the same seat at any particular point in time. 
- The system has financial transactions, meaning it should be secure and the DB ACID compliant.
- Assume traffic will spike on popular/much-awaited movie releases and the seats would fill up pretty fast, so the service should be scalable and highly available to keep up with the surge in traffic.


### Design Considerations
1. Assume that our service doesn't require authentication.
2. No handling of partial ticket orders. Either users get all the tickets they want or they get nothing.
3. Fairness is mandatory
4. To prevent system abuse, restrict users from booking more than 10 seats at a time.

## 2. Capacity Estimation

> **Traffic estimates:** 3 billion page views/month,  selling 10 million tickets a month
 
> **Storage estimates:**
500 cities, on average each city has 10 cinemas, each with 300 seats, 3 shows daily.

Let's assume each seat booking needs 50 bytes (IDs, NumberOfSeats, ShowID, MovieID, SeatNumbers, SeatStatus, Timestamp, etc) to store in the DB.
We need to store information about movies and cinemas; assume another 50 bytes.

So to store all data about all shows of all cinemas of all cities for a day

```
        500 cities * 10 cinemas * 300 seats * 3 shows * (50 + 50) bytes = 450 MB / day
```
To store data for 5 years, we'd need around
```
    450 MB/day * 365 * 5 = 821.25 GB
```

## 3. System APIs
Let's use REST APIs to expose the functionality of our service.


### Searching movies
```python
search_movies(
    api_dev_key: str,      #  The API developer key. This will be used to throttle users based on their allocated quota.
    keyword: str,          # Keyword to search on.
    city: str,             # City to filter movies by.
    lat_long: str,         # Latitude and longitude to filter by.
    radius: int,           # Radius of the area in which we want to search for events.
    start_date: datetime,  # Filter with a starting datetime.
    end_date: datetime,    # Filter with an ending datetime.
    postal_code: int,      # Filter movies by postal code / zipcode.
    include_spell_check,   # (Enum: yes or no)
    result_per_page: int   # number of results to return per page. Max = 30.
    sorting_order: str     # Sorting order of the search result. Allowable values: 'name,asc', 'name,desc', 'date,asc', 'date, desc', 'distance,asc', 'name,date,asc', 'name,date,desc'
)

```
Returns: (JSON)
```json
[
  {
    "MovieID": 1,
    "ShowID": 1,
    "Title": "Cars 2",
    "Description": "About cars",
    "Duration": 120,
    "Genre": "Animation",
    "Language": "English",
    "ReleaseDate": "8th Oct. 2014",
    "Country": USA,
    "StartTime": "14:00",
    "EndTime": "16:00",
    "Seats": 
    [
      {  
        "Type": "Regular"
        "Price": 14.99
        "Status: "Almost Full"
      },
      {  
        "Type": "Premium"
        "Price": 24.99
        "Status: "Available"
      }
    ]
  },
  {
    "MovieID": 1,
    "ShowID": 2,
    "Title": "Cars 2",
    "Description": "About cars",
    "Duration": 120,
    "Genre": "Animation",
    "Language": "English",
    "ReleaseDate": "8th Oct. 2014",
    "Country": USA,
    "StartTime": "16:30",
    "EndTime": "18:30",
    "Seats": 
    [
        {  
          "Type": "Regular"
          "Price": 14.99
          "Status: "Full"
      },
        {  
          "Type": "Premium"
        "Price": 24.99
        "Status: "Almost Full"
      }
    ]
  },
 ]
 ```
 ### Reserving Seats
 ```python
reserve_seats(
    api_dev_key: str,  # API developer key.
    session_id: str,   # User Session ID to track this reservation. Once the reservation time of 5 minutes expires, user's reservation on the server will be removed using this ID.
    movie_id: str,         # Movie to reserve.
    show_id:  str,         # Show to reserve.
    seats_to_reserve: List(int)  # An array containing seat IDs to reserve.
)
```

Returns: (JSON)
```
    The status of the reservation, which would be one of the following:
        1. Reservation Successful,
        2. Reservation Failed - Show Full
        3. Reservation Failed - Retry, as other users are holding reserved seats.
```